In [ ]:
import sys
!{sys.executable} -m pip install kfp==1.4.0 kfp-server-api==1.2.0 --user >/dev/null

## Importing Libraries

In [ ]:
import json
import kfp
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

import os
import json
import kfp
import string
import random
import kfp.dsl as dsl
import kfp.compiler as compiler
from kfp import components
from kubernetes import client as k8s_client

from dkube.sdk.api import DkubeApi
from dkube.sdk.rsrcs import DkubeCode
from dkube.sdk.rsrcs import DkubeDataset
from dkube.sdk.rsrcs import DkubeModel


## Loading Components

In [ ]:
dkube_preprocessing_op = components.load_component_from_file("/mnt/dkube/pipeline/components/preprocess/component.yaml")
dkube_training_op = components.load_component_from_file("/mnt/dkube/pipeline/components/training/component.yaml")
dkube_serving_op  = components.load_component_from_file("/mnt/dkube/pipeline/components/serving/component.yaml")

In [ ]:
user = os.getenv('USERNAME')
token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
client = kfp.Client(host=os.getenv("KF_PIPELINES_ENDPOINT"), existing_token=token, namespace=os.getenv("USERNAME"))

api = DkubeApi(token=token)

## Create Retraining model

In [ ]:
try:
    model = DkubeModel(user, name='mm-retrained-model')
    model.update_model_source(source='dvs')
    api.create_model(model)

except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

In [ ]:
training_program = 'mm-demo'
input_training_dataset = ['mm-demo-groundtruth']
input_dataset_mount = ['/data/GT-predict']
output_dataset = 'mm-training-data'
output_mount_path = ['/retrain-data']
model = 'mm-training-model'
output_model = 'mm-retrained-model'

In [ ]:
preprocessing_script = 'python insurance/modeltraining/retrain-preprocessing.py'
training_script = 'python insurance/modeltraining/retraining.py'

## Pipeline Definition

In [ ]:
@kfp.dsl.pipeline(
    name='retraining-pipeline',
    description='insurance-retraining-pl'
)
def insurance_retraining_pipeline(token):

    
    preprocessing = dkube_preprocessing_op(
                                    auth_token=str(token),
                                    container=json.dumps({"image": "ocdr/d3-datascience-sklearn:v0.23.2-1"}),
                                    program=str(training_program),
                                    datasets = json.dumps(input_training_dataset),
                                    input_dataset_mounts = json.dumps(input_dataset_mount),
                                    run_script=str(preprocessing_script),
                                    outputs=json.dumps([str(output_dataset)]),
                                    output_mounts=json.dumps(output_mount_path)).set_display_name("data-generation")
  
    
    retraining      = dkube_training_op(container=json.dumps({"image": "docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}),
                                    framework="sklearn", version="0.23.2",
                                    program=str(training_program), run_script=training_script,
                                    datasets=json.dumps([str(output_dataset)]),
                                    input_dataset_mounts=json.dumps(output_mount_path),
                                    models = json.dumps([str(model)]),
                                    input_model_mounts = '["/model"]',
                                    outputs=json.dumps([str(output_model)]),
                                    output_mounts='["/mm/retrained-model"]',
                                    auth_token=token).after(preprocessing).set_display_name("retraining")
    model_versioning = dkube_training_op(
                                    auth_token=str(token),
                                    framework="sklearn", version="0.23.2",
                                    container=json.dumps({"image": "ocdr/d3-datascience-sklearn:v0.23.2-1"}),
                                    program=str(training_program),
                                    models = json.dumps([str(output_model)]),
                                    input_model_mounts = '["/mm/retrained-model"]',
                                    outputs=json.dumps([str(model)]),
                                    output_mounts='["/models"]',
                                    run_script="cp -r /mm/retrained-model/* /models/").after(retraining).set_display_name("versioning")

    serving     = dkube_serving_op(model=model_versioning.outputs['artifact'], device='cpu', 
                                    serving_image=json.dumps({"image": "ocdr/sklearnserver:0.23.2"}),
                                    transformer_image =json.dumps({"image": "docker.io/ocdr/d3-datascience-sklearn:v0.23.2"}),
                                    transformer_project=str(training_program),
                                    transformer_code='insurance/modeltraining/transformer.py', auth_token=token).after(model_versioning).set_display_name("serving")
    

In [ ]:
client.create_run_from_pipeline_func(insurance_retraining_pipeline, arguments={'token':token})